# LSTM con WIKI

Modelo básico con los embeddings de fasttext

In [1]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random

np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)
X_test = tokenizer.texts_to_sequences(text_test)
max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)
X_test = pad_sequences(X_test, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [3]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Wikipedia Embeddings

Me quedo sólo con los embeddings de unigramas...

In [4]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "wiki.es.vec")) as f:
    for i, line in enumerate(f):
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
            word_to_vec[word] = vec
        except:
            continue
        
        
embedding_size = len(word_to_vec["hola"])

In [5]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
embedding_matrix.shape

(200000, 300)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import CuDNNLSTM, Dropout
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100))
model.add(Dropout(0.65))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, 
          validation_data=(X_dev, y_dev), 
          epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 2s 555us/step - loss: 0.6631 - acc: 0.6189 - val_loss: 0.6587 - val_acc: 0.5760
Epoch 2/20
4500/4500 [==============================] - 1s 281us/step - loss: 0.6056 - acc: 0.6887 - val_loss: 0.6052 - val_acc: 0.6640
Epoch 3/20
4500/4500 [==============================] - 1s 281us/step - loss: 0.5585 - acc: 0.7256 - val_loss: 0.5861 - val_acc: 0.7000
Epoch 4/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.5382 - acc: 0.7453 - val_loss: 0.5757 - val_acc: 0.7100
Epoch 5/20
4500/4500 [==============================] - 1s 283us/step - loss: 0.5151 - acc: 0.7573 - val_loss: 0.5728 - val_acc: 0.7140
Epoch 6/20
4500/4500 [==============================] - 1s 278us/step - loss: 0.4979 - acc: 0.7656 - val_loss: 0.5526 - val_acc: 0.7240
Epoch 7/20
4500/4500 [==============================] - 1s 289us/step - loss: 0.4870 - acc: 0.7740 - val_loss: 0.5557 - val_acc: 0.7260
E

In [8]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 101us/step
Loss           : 0.5427
Accuracy       : 0.7480
Precision(1)   : 0.7143
Precision(1)   : 0.7754
Precision(avg) : 0.7448

Recall(1)      : 0.7207
Recall(0)      : 0.7698
Recall(avg)    : 0.7453

F1(1)          : 0.7175
F1(0)          : 0.7726
F1(avg)        : 0.7450


Evaluación sobre test
1600/1600 [==============================] - 0s 97us/step
Loss           : 0.6044
Accuracy       : 0.6956
Precision(1)   : 0.6297
Precision(1)   : 0.7428
Precision(avg) : 0.6862

Recall(1)      : 0.6364
Recall(0)      : 0.7372
Recall(avg)    : 0.6868

F1(1)          : 0.6330
F1(0)          : 0.7400
F1(avg)        : 0.6865


## Bidirectional LSTM

In [9]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(CuDNNLSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           60000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               321600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 60,347,457
Trainable params: 347,457
Non-trainable params: 60,000,000
__________________________________________________________

In [10]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 138us/step
Loss           : 0.5298
Accuracy       : 0.7700
Precision(1)   : 0.7399
Precision(1)   : 0.7942
Precision(avg) : 0.7671

Recall(1)      : 0.7432
Recall(0)      : 0.7914
Recall(avg)    : 0.7673

F1(1)          : 0.7416
F1(0)          : 0.7928
F1(avg)        : 0.7672


Evaluación sobre test
1600/1600 [==============================] - 0s 137us/step
Loss           : 0.6587
Accuracy       : 0.6969
Precision(1)   : 0.6296
Precision(1)   : 0.7459
Precision(avg) : 0.6878

Recall(1)      : 0.6439
Recall(0)      : 0.7340
Recall(avg)    : 0.6890

F1(1)          : 0.6367
F1(0)          : 0.7399
F1(avg)        : 0.6883
